# **SVP plus plus**

In [ ]:
import pandas as pd
from collections import defaultdict
import random
import math
from tqdm import tqdm 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

## **1. Dùng thư viện**

### **Load data**

In [ ]:
# import zipfile

# # Đường dẫn đến file zip
# zip_path = '../Data_PPS.zip'

# # Mở và giải nén file zip
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall()

train_df = pd.read_csv('Data_PPS/train_df.csv')
test_df = pd.read_csv('Data_PPS/test_df.csv')

In [ ]:
tags = pd.read_csv('../ml-25m/tags.csv')
# Lọc tags: chỉ giữ lại userId có trong train_df
valid_user_ids = set(train_df['userId'])
tags = tags[tags['userId'].isin(valid_user_ids)]

# Tạo các cặp từ train_df và tags
train_pairs = set(zip(train_df['userId'], train_df['movieId']))
tagged_pairs = set(zip(tags['userId'], tags['movieId']))

# Gộp cả rating lẫn tag (giả sử tag là 1 dạng feedback gián tiếp)
combined_pairs = train_pairs.union(tagged_pairs)

# Tập đánh giá rõ ràng (có rating)
explicit_df = train_df[['userId', 'movieId', 'rating']]

# Các cặp chỉ có tag, chưa từng rating → gán rating giả định
tag_only_pairs = tagged_pairs - train_pairs
tag_only_df = pd.DataFrame(list(tag_only_pairs), columns=['userId', 'movieId'])
tag_only_df['rating'] = np.mean([0.5, 5.0])  # hoặc np.mean([0.5, 5.0]) = 2.75

# Gộp lại làm train data mới
train_tags_df = pd.concat([explicit_df, tag_only_df], ignore_index=True)

In [ ]:
print(train_df.shape)
print(train_tags_df.shape)

(1764200, 8)
(1834876, 3)


In [ ]:
train_tags_df.nunique()

userId      1000
movieId    56984
rating        11
dtype: int64

### **Import Surprise**

In [ ]:
from surprise import Dataset, SVDpp, accuracy, Reader
from surprise.accuracy import mae

# Định nghĩa Reader
reader = Reader(rating_scale=(0.5, 5))

# Tạo trainset và testset từ DataFrame
trainset = Dataset.load_from_df(train_tags_df[['userId', 'movieId', 'rating']], reader=reader).build_full_trainset()
testset = list(test_df[['userId', 'movieId', 'rating']].itertuples(index=False, name=None))

In [ ]:
# ======= Khởi tạo và huấn luyện mô hình =======
model_svdpp = SVDpp(n_epochs=5, n_factors=50, verbose=True)
model_svdpp.fit(trainset)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4


In [ ]:
# ======= Đánh giá trên tập huấn luyện =======
train_predictions = model_svdpp.test(trainset.build_testset()[:10000])
train_rmse = accuracy.rmse(train_predictions, verbose=True)
train_mae = accuracy.mae(train_predictions, verbose=True)

RMSE: 0.7074
MAE:  0.5365


In [ ]:
# ======= Đánh giá trên tập kiểm tra =======
test_predictions = model_svdpp.test(testset)
test_rmse = accuracy.rmse(test_predictions, verbose=True)
test_mae = accuracy.mae(test_predictions, verbose=True)

RMSE: 0.8520
MAE:  0.6408


### **Inference**

In [ ]:
def recommend_movies(user_id, model, movies_df, ratings_df, top_n=10):
    # Lấy danh sách tất cả movieId từ movies_df
    all_movie_ids = movies_df['movieId'].unique()
    
    # Lấy danh sách movieId mà người dùng đã đánh giá
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
    
    # Lấy danh sách movieId chưa được người dùng đánh giá
    unrated_movies = [mid for mid in all_movie_ids if mid not in rated_movies]
    
    # Dự đoán điểm cho các phim chưa đánh giá
    predictions = []
    for movie_id in unrated_movies:
        pred = model.predict(user_id, movie_id)
        predictions.append({
            'movieId': movie_id,
            'score': pred.est
        })
    
    # Chuyển predictions thành DataFrame
    pred_df = pd.DataFrame(predictions)
    
    # Kết hợp với movies_df để lấy tiêu đề phim
    recommended = pred_df.merge(movies_df, on='movieId', how='left')
    
    # Lấy rating thực tế (nếu có) từ ratings_df
    recommended = recommended.merge(
        ratings_df[ratings_df['userId'] == user_id][['movieId', 'rating']],
        on='movieId',
        how='left'
    )
    
    # Sắp xếp theo score giảm dần và lấy top_n
    recommended = recommended.sort_values(by='score', ascending=False).head(top_n)
    
    # Sắp xếp lại cột: movieId, title, score, rating
    recommended = recommended[['movieId', 'title', 'genres', 'score']]
    
    return recommended

In [ ]:
ratings = pd.read_csv("../ml-25m/ratings.csv")
movies = pd.read_csv('../ml-25m/movies.csv')
user_id = 95133
top_n = 10
recommended_movies = recommend_movies(user_id, model_svdpp, movies, ratings, top_n)
display(recommended_movies)

,movieId,title,genres,score
44000,170705,Band of Brothers (2001),Action|Drama|War,4.198430
39030,159817,Planet Earth (2006),Documentary,4.077804
44148,171011,Planet Earth II (2016),Documentary,4.053339
1031,1201,"Good, the Bad and the Ugly, The (Buono, il bru...",Action|Adventure|Western,4.030876
44380,171495,Cosmos,(no genres listed),4.015286
7877,26082,Harakiri (Seppuku) (1962),Drama,4.001897
6069,6787,All the President's Men (1976),Drama|Thriller,3.960995
9914,44555,"Lives of Others, The (Das leben der Anderen) (...",Drama|Romance|Thriller,3.957431
39233,160289,O.J.: Made in America (2016),Documentary,3.953772
1046,1233,"Boot, Das (Boat, The) (1981)",Action|Drama|War,3.945765


## **2. Implement**

In [2]:
# Đọc file 
ratings = pd.read_csv('/kaggle/input/movie-len-25m/ratings.csv')
movies = pd.read_csv('/kaggle/input/movie-len-25m/movies.csv')
tags = pd.read_csv('/kaggle/input/movie-len-25m/tags.csv')

In [ ]:
df = pd.merge(ratings, movies, on='movieId')

# Lọc 1000 người dùng đánh giá nhiều nhất
sample_users = df['userId'].value_counts().head(1000).index
sample_df = df[df['userId'].isin(sample_users)].copy()

# Mã hóa userId và title thành chỉ số liên tục
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
sample_df['user_idx'] = user_encoder.fit_transform(sample_df['userId'])
sample_df['movie_idx'] = movie_encoder.fit_transform(sample_df['movieId'])

# Chia dữ liệu thành tập train, test, val, đảm bảo mỗi user đều có mặt ở cả hai tập
train_val_df, test_df = train_test_split(
    sample_df, 
    test_size=0.2, 
    random_state=42,
    stratify=sample_df["user_idx"]  
)

train_df, val_df = train_test_split(
    train_val_df, 
    test_size=0.1, 
    random_state=42,
    stratify=train_val_df["user_idx"]
) 

#### **Tạo data train/test/eval**

**Explicit feedback**

- Trích xuất `[user_idx, movie_idx, rating]` cho từng tập.

- Đưa về định dạng numpy để chuẩn bị cho mô hình SVDpp và giúp truy cập nhanh, hiệu quả khi huấn luyện.

In [ ]:
# Tạo dữ liệu train/test/val cho ratings (đã encode)
rating_train = train_df[['user_idx', 'movie_idx', 'rating']].to_numpy()
rating_val = val_df[['user_idx', 'movie_idx', 'rating']].to_numpy()
rating_test = test_df[['user_idx', 'movie_idx', 'rating']].to_numpy()

**Implicit feedback**

- Xây dựng tập dữ liệu từ hành vi gắn tag để bổ sung thông tin về sở thích người dùng.

- Cách tạo data implicit feedback từ tags:

    - Tạo cặp (userId, movieId) có trong data train, thì tag tương ứng cũng có những cặp này. 
    
    - Sau đó mã hóa thành (user_idx, movie_idx) để đồng bộ với encoding từ explicit feedback.
    
    - Tạo một dictionary implicit: `user_idx -> set(movie_idx)` để biểu diễn mối quan hệ ngầm user đã tương tác với những phim nào.

In [ ]:
# Tạo cặp (userId, movieId) xuất hiện trong train
train_pairs = set(zip(train_df['userId'], train_df['movieId']))

# Lọc tags tương ứng với (userId, movieId) trong train
train_tags = tags[
    tags.apply(lambda x: (x['userId'], x['movieId']) in train_pairs, axis=1)
].copy()

# Gán user_idx và movie_idx cho tập tag
train_tags['user_idx'] = user_encoder.transform(train_tags['userId']).astype(int)
train_tags['movie_idx'] = movie_encoder.transform(train_tags['movieId']).astype(int)

# Tạo implicit feedback từ tags
implicit_train = defaultdict(set)
for row in train_tags.itertuples(index=False):
    implicit_train[row.user_idx].add(row.movie_idx)

### **Thuật toán SVD++**

#### **1. Công thức dự đoán rating và ý tưởng**

Giả sử $R_{n \times m}$ là ma trận đánh giá gồm $n$ người dùng và $m$ mặt hàng. Mỗi phần tử $r_{ui}$ là đánh giá của người dùng $u$ cho mặt hàng $i$.

Công thức dự đoán rating trong mô hình **SVD++** là:

$$
\hat{r}_{ui} = \mu + b_u + b_i + q_i^T \left(p_u + \frac{1}{\sqrt{|R(u)|}} \sum_{j \in R(u)} y_j \right)
$$

Trong đó:

- $\mu$: trung bình các rating toàn bộ hệ thống  
- $b_u$, $b_i$: độ lệch riêng của người dùng $u$ và movie $i$  
- $p_u$, $q_i$: vector tiềm ẩn đại diện cho người dùng và movie
- $y_j$: vector phản hồi ngầm của movie $j$
- $R(u)$: tập các movie mà người dùng $u$ đã đánh giá

Ý tưởng của SVD++ là kết hợp **cả thông tin explicit (rating)** và **implicit feedback (dạng hành vi như gắn tag, click, xem phim, v.v.)** để cải thiện độ chính xác khi dự đoán.

#### **2. Hàm mất mát (Loss function)**

Hàm mất mát của mô hình SVD++ được định nghĩa như sau:

$$
\sum_{r_{ui} \in R} \left[ \left(r_{ui} - \mu - b_u - b_i - q_i^T \left(p_u + \frac{1}{\sqrt{|R(u)|}} \sum_{j \in R(u)} y_j \right) \right)^2 + \lambda_1 (b_u^2 + b_i^2) + \lambda_2 ( \|p_u\|^2 + \|q_i\|^2 ) \right]
$$

Giải thích các thành phần:

- Thành phần đầu là **sai số bình phương** giữa rating thực tế và rating dự đoán.

- Các biểu thức chứa $\lambda_1$ và $\lambda_2$ là **regularization**:

  - $\lambda_1$: điều chỉnh độ lệch (bias)

  - $\lambda_2$: điều chỉnh độ lớn của vector tiềm ẩn

Regularization giúp giảm **overfitting**, nhất là khi dữ liệu đánh giá bị **thưa** (sparse). Nó hạn chế mô hình "học thuộc" toàn bộ dữ liệu train và giúp tổng quát hóa tốt hơn trên tập validation.

#### **3. Quy tắc cập nhật tham số bằng SGD**

Với mỗi mini-batch, chọn ngẫu nhiên các mẫu và cập nhật các tham số $b_u, b_i, p_u, q_i, y_j$ dựa trên sai số $e_{ui} = r_{ui} - \hat{r}_{ui}$ theo các công thức sau:

- $
b_u \leftarrow b_u + \gamma \cdot (e_{ui} - \lambda_1 \cdot b_u)
$

- $
b_i \leftarrow b_i + \gamma \cdot (e_{ui} - \lambda_1 \cdot b_i)
$

- $
q_i \leftarrow q_i + \gamma \cdot \left( e_{ui} \cdot \left( p_u + \frac{1}{\sqrt{|R(u)|}} \sum_{j \in R(u)} y_j \right) - \lambda_2 \cdot q_i \right)
$

- $
p_u \leftarrow p_u + \gamma \cdot (e_{ui} \cdot q_i - \lambda_2 \cdot p_u)
$

- $
\forall j \in R(u): \quad y_j \leftarrow y_j + \gamma \cdot \left( \frac{e_{ui}}{\sqrt{|R(u)|}} \cdot q_i - \lambda_2 \cdot y_j \right)
$

Trong đó:

- $\gamma$: learning rate  
- $\lambda_1, \lambda_2$: hệ số regularization  
- $R(u)$: tập các movie mà user $u$ đã đánh giá  

Mỗi lần cập nhật là một bước tiến nhỏ giúp mô hình tối ưu dần các tham số để giảm sai số dự đoán.


In [ ]:
class SVDPP:
    """
    Mô hình SVD++.

    ratings: list các bộ (user_idx, movie_idx, rating)
    implicit_feedback: dict {user_idx: set(movie_idx)} 
    num_users, num_items: tổng số user/item
    n_factors: số chiều của vector tiềm ẩn
    lr_all: learning rate
    reg_all: hệ số regularization mặc định
    reg1, reg2: regularization cho bias và vector
    epochs: số vòng lặp huấn luyện
    """
    def __init__(self, 
                 ratings, implicit_feedback, num_users, num_items, 
                 n_factors=20, 
                 lr_all=0.007, 
                 reg_all=0.02, 
                 reg1=None, reg2=None,
                 epochs=20):
        self.ratings = ratings  
        self.Nu = implicit_feedback  
        self.n_factors = n_factors
        self.lr_all = lr_all
        self.reg_all = reg_all
        self.reg1 = reg1 if reg1 is not None else reg_all
        self.reg2 = reg2 if reg2 is not None else reg_all
        self.epochs = epochs

        self.num_users = num_users
        self.num_items = num_items

        self.bu = np.zeros(num_users)
        self.bi = np.zeros(num_items)

        self.pu = np.random.normal(0, 0.1, (num_users, n_factors))
        self.qi = np.random.normal(0, 0.1, (num_items, n_factors))

        # yj must cover all item indices that appear in implicit feedback
        all_implicit_items = set()
        for items in implicit_feedback.values():
            all_implicit_items.update(items)
        self.yj = np.random.normal(0, 0.1, (num_items, n_factors))  # Pre-allocate full matrix

        self.global_mean = np.mean([r for _, _, r in ratings])

    def predict(self, u, i, sum_yj_u=None):
        """
        Dự đoán rating của user u cho item i.
        Nếu đã tính trước sum_yj_u thì truyền vào để tăng tốc.
        """
        if sum_yj_u is None:
            implicit_items = self.Nu.get(u, set())
            sqrt_Nu = math.sqrt(len(implicit_items)) if implicit_items else 1.0
            sum_yj_u = np.sum(self.yj[list(implicit_items)], axis=0) / sqrt_Nu if implicit_items else np.zeros(self.n_factors)

        interaction = self.pu[u] + sum_yj_u
        return self.global_mean + self.bu[u] + self.bi[i] + np.dot(interaction, self.qi[i])

    def train(self, val_set=None, early_stopping=False, patience=3):
        """
        Huấn luyện mô hình bằng SGD.

        val_set: tập validation (nếu có)
        early_stopping: optional
        patience: số vòng cho phép không cải thiện trước khi dừng
        """
        best_val_rmse = float('inf')
        patience_counter = 0

        for epoch in range(self.epochs):
            random.shuffle(self.ratings)

            # Precompute sum_yj for each user
            sum_yj_dict = {}
            for u in range(self.num_users):
                implicit_items = self.Nu.get(u, set())
                if implicit_items:
                    sqrt_Nu = math.sqrt(len(implicit_items))
                    sum_yj_dict[u] = np.sum(self.yj[list(implicit_items)], axis=0) / sqrt_Nu
                else:
                    sum_yj_dict[u] = np.zeros(self.n_factors)

            for u, i, r in tqdm(self.ratings, desc=f"Training Epoch {epoch+1}", leave=False):
                u = int(u)
                i = int(i)
                sum_yj_u = sum_yj_dict[u]
                pred = self.predict(u, i, sum_yj_u)
                err = r - pred

                # Update biases
                self.bu[u] += self.lr_all * (err - self.reg1 * self.bu[u])
                self.bi[i] += self.lr_all * (err - self.reg1 * self.bi[i])

                # Backup current vectors
                qi_i = self.qi[i].copy()
                pu_u = self.pu[u].copy()

                # Update pu and qi
                self.pu[u] += self.lr_all * (err * qi_i - self.reg2 * pu_u)
                self.qi[i] += self.lr_all * (err * (pu_u + sum_yj_u) - self.reg2 * qi_i)

                # Update yj
                implicit_items = self.Nu.get(u, set())
                if implicit_items:
                    sqrt_Nu = math.sqrt(len(implicit_items))
                    for j in implicit_items:
                        self.yj[j] += self.lr_all * (err * qi_i / sqrt_Nu - self.reg2 * self.yj[j])

            # Evaluate RMSE
            train_rmse = self.rmse(self.ratings, sum_yj_dict)
            if val_set is not None:
                val_rmse = self.rmse(val_set)
                print(f"Epoch {epoch+1} - Train RMSE: {train_rmse:.4f} | Val RMSE: {val_rmse:.4f}")
            
                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if early_stopping and patience_counter >= patience:
                        print(f"Early stopping at epoch {epoch+1}!")
                        break
            else:
                print(f"Epoch {epoch+1} - Train RMSE: {train_rmse:.4f}")

    def rmse(self, dataset, precomputed_sum_yj=None):
        """
        Tính RMSE của mô hình trên tập dữ liệu.
        """
        se = 0.0
        for u, i, r in tqdm(dataset, desc=f"\tEvaluate {len(dataset)}", leave=False):
            u = int(u)
            i = int(i)
            sum_yj_u = precomputed_sum_yj[u] if precomputed_sum_yj else None
            pred = self.predict(u, i, sum_yj_u)
            se += (r - pred) ** 2
        return math.sqrt(se / len(dataset))
    
    def mae(self, dataset, precomputed_sum_yj=None):
        """
        Tính MAE của mô hình trên tập dữ liệu.
        """
        ae = 0.0
        for u, i, r in tqdm(dataset, desc=f"\tEvaluate {len(dataset)}", leave=False):
            u = int(u)
            i = int(i)
            sum_yj_u = precomputed_sum_yj[u] if precomputed_sum_yj else None
            pred = self.predict(u, i, sum_yj_u)
            ae += abs(r - pred)
        return ae / len(dataset)

In [ ]:
num_users = sample_df['user_idx'].nunique()
num_items = sample_df['movie_idx'].nunique()

# Train SVD++
model = SVDPP(
    rating_train, implicit_train, num_users, num_items, 
    n_factors=50, 
    lr_all=0.007, 
    reg_all=0.015,
    reg1=0.015,
    reg2=0.02,
    epochs=35
)
model.train(val_set=rating_val, early_stopping=True)

Epoch 1 - Train RMSE: 0.8105 | Val RMSE: 0.8421


Epoch 2 - Train RMSE: 0.7223 | Val RMSE: 0.8360


Epoch 3 - Train RMSE: 0.5228 | Val RMSE: 0.8460


Epoch 4 - Train RMSE: 0.3369 | Val RMSE: 0.8529


Epoch 5 - Train RMSE: 0.2153 | Val RMSE: 0.8553
Early stopping at epoch 5!


In [7]:
print(
    f"\nRMSE | MAE\n"
    f"Train: {model.rmse(rating_train):.4f} | {model.mae(rating_train):.4f}\n"
    f"Val  : {model.rmse(rating_val):.4f} | {model.mae(rating_val):.4f}\n"
    f"Test : {model.rmse(rating_test):.4f} | {model.mae(rating_test):.4f}"
)


RMSE | MAE
Train: 0.2175 | 0.1102
Val  : 0.8553 | 0.6537
Test : 0.8510 | 0.6515


### **Inference**

In [ ]:
def recommend_top_k_df(user_id, model, user_encoder, train_df, movie_df, k=10):
    """
    Trả về top-k phim được đề xuất cho user_id dựa trên mô hình đã huấn luyện.

    user_id: ID người dùng gốc (chưa encode)
    model: mô hình SVDPP đã huấn luyện
    user_encoder: bộ mã hóa userId → user_idx
    train_df: tập dữ liệu train chứa user_idx, movie_idx
    movie_df: thông tin phim (chứa movieId, title, genres)
    k: số lượng phim đề xuất

    return: DataFrame chứa top-k phim đề xuất với thông tin kèm theo
    """
    # Kiểm tra user có trong tập train không
    if user_id not in  train_df['userId'].values:
        print(f"User ID {user_id} không có trong tập train.")
        return pd.DataFrame()

    # Mã hóa user_id sang user_idx
    user_idx = user_encoder.transform([user_id])[0]

    # Lọc phim user đã xem
    rated_movie_idxs = set(train_df[train_df['user_idx'] == user_idx]['movie_idx'])

    # Dự đoán cho các phim chưa xem và lấy top-k
    all_movie_idxs = np.arange(model.num_items)
    candidates = [
        (i, model.predict(user_idx, i))
        for i in all_movie_idxs if i not in rated_movie_idxs
    ]
    top_k = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]

    # Tạo DataFrame từ kết quả
    top_k_df = pd.DataFrame(top_k, columns=['movie_idx', 'score_rating_predict'])
    movie_info = movie_df[['movie_idx', 'movieId', 'title', 'genres']].drop_duplicates()
    result_df = pd.merge(top_k_df, movie_info, on='movie_idx', how='left')

    result_df = result_df[['movieId', 'title', 'genres', 'score_rating_predict']]
    result_df['score_rating_predict'] = result_df['score_rating_predict'].round(3)

    return result_df

In [9]:
recommendations = recommend_top_k_df(
    user_id=95133,
    model=model,
    user_encoder=user_encoder,
    train_df=train_df,
    movie_df=sample_df,  
    k=10
)
display(recommendations)


,movieId,title,genres,score_rating_predict
0,4235,Amores Perros (Love's a Bitch) (2000),Drama|Thriller,4.265
1,71462,"Cove, The (2009)",Documentary,4.240
2,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.238
3,2068,Fanny and Alexander (Fanny och Alexander) (1982),Drama|Fantasy|Mystery,4.217
4,3462,Modern Times (1936),Comedy|Drama|Romance,4.129
5,53123,Once (2006),Drama|Musical|Romance,4.089
6,6041,Amen. (2002),Drama,4.084
7,1280,Raise the Red Lantern (Da hong deng long gao g...,Drama,4.065
8,6818,Come and See (Idi i smotri) (1985),Drama|War,4.018
9,7396,Scenes From a Marriage (Scener ur ett äktenska...,Drama,4.009
